In [6]:
from pony.orm import db_session
from CGRdb.database import db
from CGRtools import smiles
from CGRdb.database import Substance
from multiprocess import Process, Queue
from tqdm import tqdm
import zipfile

In [7]:
# If database is not empty and you want to erase it, uncomment this
from CGRdb.database import db
from pony.orm import db_session
from CGRdb.database.config import Config
db.bind(provider='postgres', user='postgres', host='localhost', password="example", database='test',
        port=5432)
db.generate_mapping(check_tables=False, create_tables=True)
db.drop_all_tables(with_all_data=True)
db.commit()
db.disconnect()
db.unbind()

In [8]:
# Connection to the database
db.bind(provider='postgres', user='postgres', host='localhost', password="example", database='test',
        port=5432)
db.generate_mapping(create_tables=True)

In [9]:
# initialize parameres for fingerprint and LSH
from CGRdb.database.config import Config
db.execute("Create extension if not exists intarray;")
Config(key="fingerprint",value={"min_radius":1, "max_radius":6, "length":2048,
                 "number_active_bits":2, "number_bit_pairs":4, "include_hydrogens":False})
Config(key="lsh_num_permute",value=64)
Config(key="lsh_threshold",value=0.7)
db.commit()
db.disconnect()
db.unbind()

In [10]:
def worker(q):
    db.bind(provider='postgres', user='postgres', host='localhost', password="example", database='test',
        port=5432)
    db.generate_mapping()
    while True:
        data = q.get()
        if data is None:
            break
        for i in data:
            mol, label = i
            mol = smiles(mol)
            mol.canonicalize()
            subs = [(x,None) for x in mol.split()]
            for _ in range(10):
                try:
                    with db_session():
                        Substance(subs)
                        break
                except Exception:
                    continue
    print("finished")

In [11]:
q = Queue(maxsize=30)
num_workers=20
pr = [Process(target=worker, 
              args=(q,)) for _ in range(num_workers)]
[p.start() for p in pr]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Process Process-7:
Traceback (most recent call last):
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/v6/nhck_l1546v4941cq8jn4hr00000gq/T/ipykernel_4529/1009587198.py", line 12, in worker
    mol.canonicalize()
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/standardize/molecule.py", line 43, in canonicalize
    k = self.kekule()
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/aromatics.py", line 204, in kekule
    kekule = next(self.__kekule_full(), None)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/aromatics.py", line 455, in __kekule_full
    rings, pyroles, double_bonded = self.__prepare_rings()
  

  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/standardize/molecule.py", line 44, in canonicalize
    s = self.standardize(fix_stereo=False, logging=logging)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/standardize/molecule.py", line 44, in canonicalize
    s = self.standardize(fix_stereo=False, logging=logging)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/aromatics.py", line 193, in thiele
    self._fix_stereo()  # check if any stereo centers vanished.
Traceback (most recent call last):
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/standardize/molecule.py", line 44, in canonicalize
    s = self.standardize(fix_stereo=False, logging=logging)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/standardize/molecule.py", line 44, in canonicalize
    s = self.standardize(fix_stereo=F

  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/sssr.py", line 58, in _sssr
    bonds = cls._skin_graph(bonds)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/isomorphism.py", line 96, in get_mapping
    m = next(g, None)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/stereo.py", line 707, in _chiral_tetrahedrons
    return self.__chiral_centers[0]
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/periodictable/element/element.py", line 99, in from_symbol
    element = next(x for x in Element.__subclasses__() if x.__name__ == symbol)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 481, in __exit__
    db_session._commit_or_rollback(exc_t

  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CachedMethods/__init__.py", line 67, in __get__
    value = self.func(obj)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/periodictable/element/query.py", line 202, in __eq__
    def __eq__(self, other):
  File "<string>", line 2, in commit
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 481, in __exit__
    db_session._commit_or_rollback(exc_type, exc, tb)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 4014, in get
    try: return entity._find_one_(kwargs)  # can throw MultipleObjectsFoundError
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/periodictable/groupXIII.py", line 30, in atomic_number
    @property
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/CGRtools/algorithms/stereo.py", line 838, in __stereo_axises
    for ax, env in zip(*self._s

  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 5274, in _save_created_
    if auto_pk: new_id = database._exec_sql(sql, arguments, returning_id=True,
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 5274, in _save_created_
    if auto_pk: new_id = database._exec_sql(sql, arguments, returning_id=True,
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 5274, in _save_created_
    if auto_pk: new_id = database._exec_sql(sql, arguments, returning_id=True,
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 1901, in flush
    if obj is not None: obj._save_()
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 952, in _exec_sql
    try: new_id = provider.execute(cursor, sql, arguments, returning_id)
KeyboardInterrupt
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packa

  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 4180, in _find_in_db_
    cursor = database._exec_sql(sql, arguments)
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 5437, in _save_
    if status == 'created': obj._save_created_()
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 1901, in flush
    if obj is not None: obj._save_()
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 947, in _exec_sql
    connection = cache.prepare_connection_for_query_execution()
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 5274, in _save_created_
    if auto_pk: new_id = database._exec_sql(sql, arguments, returning_id=True,
  File "/Users/timur/Work/CGRDB_new/venv/lib/python3.9/site-packages/pony/orm/core.py", line 5437, in _save_
    if status == 'created': obj._save_created_()
  Fil

In [12]:
with zipfile.ZipFile("../dataset/Chembl28.smi.zip", 'r') as zip_ref:
    zip_ref.extractall("")
with open("Chembl28.smi",) as f:
    tmp = []
    for n, row in tqdm(enumerate(f)):
        columns = row.rstrip('\n').lstrip().split(" ")
        smi = columns[0]
        idx = None
        if len(columns) > 1:
            if columns[1].startswith("|"):
                smi += columns[1]
                if len(columns) > 2:
                    idx = columns[2]
            else:
                idx = columns[1]
        if not idx:
            idx = n
        tmp.append((smi, idx))
        if len(tmp) == 1000:
            q.put(tmp)
            tmp = []
    else:
        q.put(tmp)
    for i in range(num_workers):
        q.put(None)

331999it [16:40, 331.73it/s] 


KeyboardInterrupt: 

In [13]:
db.bind(provider='postgres', user='postgres', host='localhost', password="example", database='test',
        port=5432)
db.generate_mapping()
db.create_fing_index()
db.create_sim_index()

creation of index for fingerprints
creation of index for fingerprint length
Creation of LSH


281978it [04:02, 1160.53it/s]
0it [00:00, ?it/s]

Uploading LSH tables into DB


8it [10:30, 78.84s/it]
